# Análise e Engenharia de Dados no E-commerce Brasileiro


## Introdução


No presente projeto, utilizou-se o 'Brazilian E-Commerce Public Dataset by Olist' como base para simular o papel de engenheiros de dados em uma empresa de e-commerce. O propósito é extrair insights valiosos que não apenas melhorem as operações de negócios, mas também otimizem a logística e aprimorem a experiência do cliente. Ao mergulharmos nos dados reais do setor de e-commerce brasileiro, esta iniciativa oferece uma oportunidade prática de explorar tendências, identificar padrões e, consequentemente, orientar decisões estratégicas. Além disso, destacamos a importância de análises orientadas por dados no contexto dinâmico do comércio eletrônico, evidenciando como tais abordagens podem impulsionar melhorias tangíveis e sustentáveis no desempenho empresarial.



In [1]:
%%capture
!pip install -r requirements.txt

In [2]:
import re

### Importação de bibliotecas

In [3]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

import zipfile
import pyspark
import re

import findspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import when
from pyspark.sql.functions import udf
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import BooleanType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType

### Iniciando uma sessão Spark

In [4]:
import findspark

findspark.init(r'C:\ProgramData\anaconda3\pkgs\pyspark-3.5.1-pyhd8ed1ab_0\site-packages\pyspark')
findspark.find()

spark = SparkSession.builder \
    .appName("Meu aplicativo Spark") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", 4) \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.driver.extraClassPath", "postgresql-42.7.3.jar") \
    .config("spark.memory.offHeap.size", "8g") \
    .getOrCreate()


In [5]:
spark

### Importando dataset via Kaggle API

In [6]:
# Download do conjunto de dados via API do Kaggle.\n",
!kaggle datasets download -d olistbr/brazilian-ecommerce

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
brazilian-ecommerce.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
#Descompactando arquivo recebido
arquivo_zip = 'brazilian-ecommerce.zip'

# Selecionando uma pasta chamada /csv/ para melhorar a organização.
destination_folder = 'csv/'
with zipfile.ZipFile(arquivo_zip, 'r') as zipado:
    zipado.extractall(destination_folder)

### Criação de dataframes

In [8]:
df_geolocation = spark.read.csv('csv/olist_geolocation_dataset.csv', header=True, inferSchema=True)
df_customers = spark.read.csv('csv/olist_customers_dataset.csv', header=True, inferSchema=True)
df_items = spark.read.csv('csv/olist_order_items_dataset.csv', header=True, inferSchema=True)
df_payments = spark.read.csv('csv/olist_order_payments_dataset.csv', header=True, inferSchema=True)
df_reviews = spark.read.csv('csv/olist_order_reviews_dataset.csv', header=True, inferSchema=True)
df_orders = spark.read.csv('csv/olist_orders_dataset.csv', header=True, inferSchema=True)
df_products = spark.read.csv('csv/olist_products_dataset.csv', header=True, inferSchema=True)
df_sellers = spark.read.csv('csv/olist_sellers_dataset.csv', header=True, inferSchema=True)
df_names = spark.read.csv('csv/product_category_name_translation.csv', header=True, inferSchema=True)

# Tratamento

## Dataset 'geolocation'
Geolocalização: Fornece dados de geolocalização relacionados aos clientes.
### Descrição das variáveis
![Geolocalização](img/dicionario/geolocalizacao.png)

### Limpeza e tratamento dos dados

In [9]:
print('linhas: ', df_geolocation.count())
print('colunas: ', len(df_geolocation.columns))

linhas:  1000163
colunas:  5


In [10]:
df_geolocation.describe().show()

+-------+---------------------------+------------------+-------------------+----------------+-----------------+
|summary|geolocation_zip_code_prefix|   geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|
+-------+---------------------------+------------------+-------------------+----------------+-----------------+
|  count|                    1000163|           1000163|            1000163|         1000163|          1000163|
|   mean|          36574.16646586607|-21.17615291038373| -46.39054132093608|            NULL|             NULL|
| stddev|         30549.335710320098| 5.715866308822804|  4.269748306619733|            NULL|             NULL|
|    min|                       1001| -36.6053744107061|-101.46676644931476|        * cidade|               AC|
|    max|                      99990| 45.06593318269697| 121.10539381057764|            óleo|               TO|
+-------+---------------------------+------------------+-------------------+----------------+-----------

Tipos de Dados: O conjunto de dados contém uma mistura de tipos de dados. geolocation_zip_code_prefix é do tipo int64, o que é apropriado para códigos postais. geolocation_lat (latitude) e geolocation_lng (longitude) são do tipo float64, que é adequado para coordenadas geográficas. geolocation_city e geolocation_state são do tipo objeto, indicando que provavelmente são valores de string representando nomes geográficos.
m Valores Ausentes ou Zero: Nenhuma das colunas contém valores nulos ou zero. Isso indica boa integridade dos dados para esses campos.

Valor Negativo: As colunas geolocation_lat e geolocation_lng mostram quase todos os valores como negativos (99% e 100%, respectivamente). Isso é realmente esperado para coordenadas no Brasil, já que está localizado no Hemisfério Ocidental (longitude negativa) e principalmente no Hemisfério Sul (latitude negati

Duplicados: O conjunto de dados possui um número muito grande de linhas duplicadas (261.831), o que sugere um problema de entrada de dados ou que o processo de coleta de dados capturou várias entradas para os mesmos pontos de geolocalização.oints.df



### Corrigindo nomes de cidades


In [11]:
def filtrar_cidade(data, nome_coluna):
    padrao = "[^a-zA-Z0-9\s\-\'\+]"
    df_filtrado = data.filter(col(nome_coluna).rlike(padrao))    
    return df_filtrado


In [12]:
filtrar_cidade(df_geolocation, 'geolocation_city')

DataFrame[geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string]

## Dataset 'customers'
Clientes: Contém informações sobre os clientes do sistema de comércio eletrônico.
### Descrição das variáveis
![Clientes](img/dicionario/clientes.png)

### Limpeza e tratamento dos dados

In [13]:
print('linhas: ', df_customers.count())
print('colunas: ', len(df_customers.columns))

linhas:  99441
colunas:  5


In [14]:
df_customers.show(10)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [15]:
df_customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [16]:
print(f'Contagem de valores nulos:')
for coluna in df_customers.columns:
    print('Coluna:', coluna, '| Quantidade:', df_customers.filter(df_customers[coluna].isNull()).count())

print('\n')

print(f'Contagem de valores zerados:')
for coluna in df_customers.columns:
    print('Coluna:', coluna, '| Quantidade:', df_customers.filter(df_customers[coluna] == 0).count())

print('\n')

print(f'Contagem de valores negativos:')
for coluna in df_customers.columns:
    print('Coluna:', coluna, '| Quantidade:', df_customers.filter(df_customers[coluna] < 0).count())

Contagem de valores nulos:
Coluna: customer_id | Quantidade: 0
Coluna: customer_unique_id | Quantidade: 0
Coluna: customer_zip_code_prefix | Quantidade: 0
Coluna: customer_city | Quantidade: 0
Coluna: customer_state | Quantidade: 0


Contagem de valores zerados:
Coluna: customer_id | Quantidade: 0
Coluna: customer_unique_id | Quantidade: 0
Coluna: customer_zip_code_prefix | Quantidade: 0
Coluna: customer_city | Quantidade: 0
Coluna: customer_state | Quantidade: 0


Contagem de valores negativos:
Coluna: customer_id | Quantidade: 0
Coluna: customer_unique_id | Quantidade: 0
Coluna: customer_zip_code_prefix | Quantidade: 0
Coluna: customer_city | Quantidade: 0
Coluna: customer_state | Quantidade: 0


In [17]:
df_customers = df_customers.withColumn("customer_zip_code_prefix_INT", df_customers["customer_zip_code_prefix"].cast(IntegerType()))
df_customers = df_customers.drop('customer_zip_code_prefix')
df_customers = df_customers.withColumnRenamed('customer_zip_code_prefix_INT', 'customer_zip_code_prefix')
df_customers.show()

+--------------------+--------------------+--------------------+--------------+------------------------+
|         customer_id|  customer_unique_id|       customer_city|customer_state|customer_zip_code_prefix|
+--------------------+--------------------+--------------------+--------------+------------------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|              franca|            SP|                   14409|
|18955e83d337fd6b2...|290c77bc529b7ac93...|sao bernardo do c...|            SP|                    9790|
|4e7b3e00288586ebd...|060e732b5b29e8181...|           sao paulo|            SP|                    1151|
|b2b6027bc5c5109e5...|259dac757896d24d7...|     mogi das cruzes|            SP|                    8775|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|            campinas|            SP|                   13056|
|879864dab9bc30475...|4c93744516667ad3b...|      jaragua do sul|            SC|                   89254|
|fd826e7cf63160e53...|addec96d2e059c80c...|           s

In [18]:
intervalos_cep_uf = {'AC': (69900, 69999),
                     'AL': (57000, 57999),
                     'AP': (68900, 68999),
                     'AM': ((69000, 69299), (69400, 69899)),
                     'BA': (40000, 48999),
                     'CE': (60000, 63999),
                     'DF': (70000, 73699),
                     'ES': (29000, 29999),
                     'GO': ((72800, 72999), (73700, 76799)),
                     'MA': (65000, 65999),
                     'MT': (78000, 78899),
                     'MS': (79000, 79999),
                     'MG': (30000, 39999),
                     'PA': (66000, 68899),
                     'PB': (58000, 58999),
                     'PR': (80000, 87999),
                     'PE': (50000, 56999),
                     'PI': (64000, 64999),
                     'RJ': (20000, 28999),
                     'RN': (59000, 59999),
                     'RS': (90000, 99999),
                     'RO': (76800, 76999),
                     'RR': (69300, 69399),
                     'SC': (88000, 89999),
                     'SP': (1000, 19999),
                     'SE': (49000, 49999),
                     'TO': (77000, 77999)}

def verifica_cep(cep, estado):
  intervalo_valido = f'invalido'
  intervalo = intervalos_cep_uf[estado]
  if all(isinstance(elemento, tuple) for elemento in intervalo):
    primeira, segunda = intervalo
    if primeira[0] <= cep <= primeira[1] or segunda[0] <= cep <= segunda[1]:
      intervalo_valido = f'valido'
  else:
    if intervalo[0] <= cep <= intervalo[1]:
      intervalo_valido = f'valido'
  return intervalo_valido

verificar_cep_valido_udf = udf(verifica_cep, StringType())
df_customers = df_customers.withColumn('cep_valido', verificar_cep_valido_udf(col('customer_zip_code_prefix'), col('customer_state')))
df_customers.show()

+--------------------+--------------------+--------------------+--------------+------------------------+----------+
|         customer_id|  customer_unique_id|       customer_city|customer_state|customer_zip_code_prefix|cep_valido|
+--------------------+--------------------+--------------------+--------------+------------------------+----------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|              franca|            SP|                   14409|    valido|
|18955e83d337fd6b2...|290c77bc529b7ac93...|sao bernardo do c...|            SP|                    9790|    valido|
|4e7b3e00288586ebd...|060e732b5b29e8181...|           sao paulo|            SP|                    1151|    valido|
|b2b6027bc5c5109e5...|259dac757896d24d7...|     mogi das cruzes|            SP|                    8775|    valido|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|            campinas|            SP|                   13056|    valido|
|879864dab9bc30475...|4c93744516667ad3b...|      jaragua do sul|        

In [19]:
df_customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- cep_valido: string (nullable = true)



In [20]:
df_customers = df_customers.filter(F.col("cep_valido") == "invalido")
df_customers.show()

+-----------+------------------+-------------+--------------+------------------------+----------+
|customer_id|customer_unique_id|customer_city|customer_state|customer_zip_code_prefix|cep_valido|
+-----------+------------------+-------------+--------------+------------------------+----------+
+-----------+------------------+-------------+--------------+------------------------+----------+



Tipos de dados: os tipos de dados no conjunto de dados incluem int64 para a coluna customer_zip_code_prefix e object para todas as outras colunas, como customer_id, customer_unique_id, customer_city e customer_state.

Sem valores ausentes ou negativos: nenhuma das colunas contém valores nulos ou negativos. Isso indica boa integridade de dados para esses campos.

Sem dupdicatas: não há linhas duplicadas neste segmento do conjunto de dados, o que sugere que cada entrada é única.

Diversidade de dados: Os vaamostra únicaunique para customer_state revelam que o conjunto de dados inclui clientes de uma ampla variedade de estados brasileiros, o que pode ser valioso para segmentação de mercado e análise regional.

## Dataset 'items'
Itens de Pedido : A tabela contém informações sobre itens pedidos em um sistema de comércio eletrônico. Cada linha representa um item dentro de um pedido.
### Descrição das variáveis
![Clientes](img/dicionario/itenspedidos.png)

### Limpeza e tratamento dos dados

In [21]:
print('linhas: ', df_items.count())
print('colunas: ', len(df_items.columns))

linhas:  112650
colunas:  7


In [22]:
df_items.describe().show(10)

+-------+--------------------+------------------+--------------------+--------------------+------------------+------------------+
|summary|            order_id|     order_item_id|          product_id|           seller_id|             price|     freight_value|
+-------+--------------------+------------------+--------------------+--------------------+------------------+------------------+
|  count|              112650|            112650|              112650|              112650|            112650|            112650|
|   mean|                NULL|1.1978339991122948|                NULL|                NULL|120.65373901464174|19.990319928982977|
| stddev|                NULL|0.7051240313951721|                NULL|                NULL| 183.6339280502595|15.806405412297098|
|    min|00010242fe8c5a6d1...|                 1|00066f42aeeb9f300...|0015a82c2db000af6...|              0.85|               0.0|
|    max|fffe41c64501cc87c...|                21|fffe9eeff12fcbd74...|ffff564a4f9085cd2...

In [23]:
df_items.show(10)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [24]:
df_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [25]:
print(f'Contagem de valores nulos:')
for coluna in df_items.columns:
    print('Coluna:', coluna, '| Quantidade:', df_items.filter(df_items[coluna].isNull()).count())

Contagem de valores nulos:
Coluna: order_id | Quantidade: 0
Coluna: order_item_id | Quantidade: 0
Coluna: product_id | Quantidade: 0
Coluna: seller_id | Quantidade: 0
Coluna: shipping_limit_date | Quantidade: 0
Coluna: price | Quantidade: 0
Coluna: freight_value | Quantidade: 0


In [26]:
print(f'Contagem de valores zerados:')
for coluna in df_items.columns:
    if isinstance(df_items.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_items.filter(df_items[coluna] == 0).count())

Contagem de valores zerados:
Coluna: order_id | Quantidade: 0
Coluna: order_item_id | Quantidade: 0
Coluna: product_id | Quantidade: 0
Coluna: seller_id | Quantidade: 0
Coluna: price | Quantidade: 0
Coluna: freight_value | Quantidade: 383


In [27]:
print(f'Contagem de valores negativos:')
for coluna in df_items.columns:
    if isinstance(df_items.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_items.filter(df_items[coluna] < 0).count())

Contagem de valores negativos:
Coluna: order_id | Quantidade: 0
Coluna: order_item_id | Quantidade: 0
Coluna: product_id | Quantidade: 0
Coluna: seller_id | Quantidade: 0
Coluna: price | Quantidade: 0
Coluna: freight_value | Quantidade: 0


Tipos de dados: o conjunto de dados inclui vários tipos de dados adequados para os dados que representam:

order_id, product_id e seller_id são do tipo object, que normalmente indica identificadores de string.
order_item_id é um int64, provavelmente representando um identificador numérico ou uma contagem.
shipping_limit_date é um objeto que deve estar no formato datetime porque representa a hora.
price e frete_value são float64, apropriados para representar valores monetários.
Sem valores ausentes ou negativos: não há valores nulos ou negativos em todas as colunas.

Valores Zero: A coluna frete_valor possui uma pequena porcentagem de valores zero (0,34%). Valores zero na coluna frete_valor podem sugerir frete grátis.

Duplicatas: nenhuma linha duplicada é relatada neste subconjunto do conjunto de dados.

## Dataset 'payments'
Pagamentos : Esta tabela contém informações sobre os pagamentos feitos para pedidos em e-commerce. Cada linha representa uma transação de pagamento única, detalhando como os clientes pagaram por seus pedidos.

### Descrição das variáveis
![Clientes](img/dicionario/pagamentos.png)

### Limpeza e tratamento dos dados

In [28]:
print('linhas: ', df_payments.count())
print('colunas: ', len(df_payments.columns))

linhas:  103886
colunas:  5


In [29]:
df_payments.describe().show(10)

+-------+--------------------+------------------+------------+--------------------+------------------+
|summary|            order_id|payment_sequential|payment_type|payment_installments|     payment_value|
+-------+--------------------+------------------+------------+--------------------+------------------+
|  count|              103886|            103886|      103886|              103886|            103886|
|   mean|                NULL|1.0926785129853878|        NULL|   2.853348863176944|154.10038041698792|
| stddev|                NULL|0.7065837791949958|        NULL|   2.687050673856492|  217.494063864724|
|    min|00010242fe8c5a6d1...|                 1|      boleto|                   0|               0.0|
|    max|fffe41c64501cc87c...|                29|     voucher|                  24|          13664.08|
+-------+--------------------+------------------+------------+--------------------+------------------+



In [30]:
df_payments.show(10)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [31]:
df_payments.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [32]:
print(f'Contagem de valores nulos:')
for coluna in df_payments.columns:
    print('Coluna:', coluna, '| Quantidade:', df_payments.filter(df_payments[coluna].isNull()).count())

print('\n')

print(f'Contagem de valores zerados:')
for coluna in df_payments.columns:
    print('Coluna:', coluna, '| Quantidade:', df_payments.filter(df_payments[coluna] == 0).count())

print('\n')

print(f'Contagem de valores negativos:')
for coluna in df_payments.columns:
    print('Coluna:', coluna, '| Quantidade:', df_payments.filter(df_payments[coluna] < 0).count())

Contagem de valores nulos:
Coluna: order_id | Quantidade: 0
Coluna: payment_sequential | Quantidade: 0
Coluna: payment_type | Quantidade: 0
Coluna: payment_installments | Quantidade: 0
Coluna: payment_value | Quantidade: 0


Contagem de valores zerados:
Coluna: order_id | Quantidade: 0
Coluna: payment_sequential | Quantidade: 0
Coluna: payment_type | Quantidade: 0
Coluna: payment_installments | Quantidade: 2
Coluna: payment_value | Quantidade: 9


Contagem de valores negativos:
Coluna: order_id | Quantidade: 0
Coluna: payment_sequential | Quantidade: 0
Coluna: payment_type | Quantidade: 0
Coluna: payment_installments | Quantidade: 0
Coluna: payment_value | Quantidade: 0


### Verificar se o número de parcelas de pagamento é igual a 0

In [33]:
# Filtrar os registros onde 'payment_value' é igual a 0 e 'payment_type' é igual a 'not_defined'
notdefined_0 = df_payments.filter((col("payment_value") == 0) & (col("payment_type") == "not_defined"))
df_payments = df_payments.join(notdefined_0, how='left_anti')
df_payments.show()

+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------+------------------+------------+--------------------+-------------+
+--------+------------------+------------+--------------------+-------------+



## Dataset 'reviews'
Reviews : Esta tabela contém informações sobre as avaliações deixadas pelos clientes para os pedidos que fizeram em uma plataforma de e-commerce. Cada linha representa um feedback do cliente sobre seus pedidos.
### Descrição das variáveis
![Avaliacoes](img/dicionario/avaliacoes.png)

### Limpeza e tratamento dos dados

In [34]:
print('linhas: ', df_reviews.count())
print('colunas: ', len(df_reviews.columns))

linhas:  104162
colunas:  7


In [35]:
df_reviews.describe().show(10)

+-------+------------------+--------------------+--------------------+--------------------+----------------------+--------------------+-----------------------+
|summary|         review_id|            order_id|        review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+-------+------------------+--------------------+--------------------+--------------------+----------------------+--------------------+-----------------------+
|  count|             84300|               82507|               82383|                9771|                 33264|               77186|                  77166|
|   mean|               4.5|                 0.0|   4.085132880021918|4.017178548731727E10|     8.333333333333334|                NULL|                   NULL|
| stddev|0.7071067811865476|                 0.0|  1.3828056742856079|6.337235641972761E11|     2.921186973360886|                NULL|                   NULL|
|    min|                 "|            

In [36]:
df_reviews.show(10)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                NULL|                  NULL| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                NULL|                  NULL| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                NULL|                  NULL| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                NULL|  Recebi bem antes ...| 2017-04-21 00:00:00|   

In [37]:
df_reviews.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



In [38]:
print(f'Contagem de valores nulos:')
for coluna in df_reviews.columns:
    print('Coluna:', coluna, '| Quantidade:', df_reviews.filter(df_reviews[coluna].isNull()).count())

print('\n')

print(f'Contagem de valores zerados:')
for coluna in df_reviews.columns:
    print('Coluna:', coluna, '| Quantidade:', df_reviews.filter(df_reviews[coluna] == 0).count())

print('\n')

print(f'Contagem de valores negativos:')
for coluna in df_reviews.columns:
    print('Coluna:', coluna, '| Quantidade:', df_reviews.filter(df_reviews[coluna] < 0).count())

Contagem de valores nulos:
Coluna: review_id | Quantidade: 1
Coluna: order_id | Quantidade: 2236
Coluna: review_score | Quantidade: 2380
Coluna: review_comment_title | Quantidade: 92157
Coluna: review_comment_message | Quantidade: 63079
Coluna: review_creation_date | Quantidade: 8764
Coluna: review_answer_timestamp | Quantidade: 8785


Contagem de valores zerados:
Coluna: review_id | Quantidade: 2
Coluna: order_id | Quantidade: 3
Coluna: review_score | Quantidade: 2
Coluna: review_comment_title | Quantidade: 23
Coluna: review_comment_message | Quantidade: 53
Coluna: review_creation_date | Quantidade: 0
Coluna: review_answer_timestamp | Quantidade: 0


Contagem de valores negativos:
Coluna: review_id | Quantidade: 0
Coluna: order_id | Quantidade: 0
Coluna: review_score | Quantidade: 0
Coluna: review_comment_title | Quantidade: 0
Coluna: review_comment_message | Quantidade: 0
Coluna: review_creation_date | Quantidade: 0
Coluna: review_answer_timestamp | Quantidade: 0


In [39]:
df_reviews = df_reviews.dropna()

Tipos de dados: o conjunto de dados contém várias colunas com tipos de dados de objetos que são típicos para dados de string, como IDs e texto, e int64 para valores numéricos como review_score. review_creation_date e review_answer_timestamp são objetos, que devem estar no formato data e hora porque representam a hora.

Valor ausente: as colunas review_comment_title e review_comment_message possuem uma porcentagem significativa de valores nulos, 88% e 58% respectivamente. Isso pode indicar que muitos clientes não deixam um título ou mensagem com sua avaliação, o que é comum em conjuntos de dados de avaliação. Eliminaremos esta coluna porque o número de valores ausentes é superior a 50%.

Sem valores negativos ou zero: não há valores negativos ou zero em colunas numéricas.

Duplicatas: não há linhas duplicadas no conjunto de dados, o que indica que cada linha representa uma revisão exclusiva.

## Dataset 'orders'
Pedidos : Esta tabela contém informações sobre os pedidos feitos pelos clientes em plataformas de e-commerce. Cada linha representa um pedido único feito pelos clientes.
### Descrição das variáveis
![Pedidos](img/dicionario/pedidos.png)

### Limpeza e tratamento dos dados

In [40]:
print('linhas: ', df_orders.count())
print('colunas: ', len(df_orders.columns))

linhas:  99441
colunas:  8


In [41]:
df_orders.describe().show(10)

+-------+--------------------+--------------------+------------+
|summary|            order_id|         customer_id|order_status|
+-------+--------------------+--------------------+------------+
|  count|               99441|               99441|       99441|
|   mean|                NULL|                NULL|        NULL|
| stddev|                NULL|                NULL|        NULL|
|    min|00010242fe8c5a6d1...|00012a2ce6f8dcda2...|    approved|
|    max|fffe41c64501cc87c...|ffffe8b65bbe3087b...| unavailable|
+-------+--------------------+--------------------+------------+



In [42]:
df_orders.show(10)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [43]:
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [44]:
print(f'Contagem de valores nulos:')
for coluna in df_orders.columns:
    print('Coluna:', coluna, '| Quantidade:', df_orders.filter(df_orders[coluna].isNull()).count())

print('\n')

print(f'Contagem de valores zerados:')
for coluna in df_orders.columns:
    if isinstance(df_orders.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_orders.filter(df_orders[coluna] == 0).count())

print('\n')

print(f'Contagem de valores negativos:')
for coluna in df_orders.columns:
    if isinstance(df_orders.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_orders.filter(df_orders[coluna] < 0).count())

Contagem de valores nulos:
Coluna: order_id | Quantidade: 0
Coluna: customer_id | Quantidade: 0
Coluna: order_status | Quantidade: 0
Coluna: order_purchase_timestamp | Quantidade: 0
Coluna: order_approved_at | Quantidade: 160
Coluna: order_delivered_carrier_date | Quantidade: 1783
Coluna: order_delivered_customer_date | Quantidade: 2965
Coluna: order_estimated_delivery_date | Quantidade: 0


Contagem de valores zerados:
Coluna: order_id | Quantidade: 0
Coluna: customer_id | Quantidade: 0
Coluna: order_status | Quantidade: 0


Contagem de valores negativos:
Coluna: order_id | Quantidade: 0
Coluna: customer_id | Quantidade: 0
Coluna: order_status | Quantidade: 0


In [45]:
# Removendo valores nulos
df_orders = df_orders.dropna()

# Converter colunas relacionadas a datas para o tipo de dados datetime
df_orders = df_orders.withColumn("order_purchase_timestamp", to_timestamp(col("order_purchase_timestamp"), "yyyy-MM-dd HH:mm:ss"))
df_orders = df_orders.withColumn("order_approved_at", to_timestamp(col("order_approved_at"), "yyyy-MM-dd HH:mm:ss"))
df_orders = df_orders.withColumn("order_delivered_carrier_date", to_timestamp(col("order_delivered_carrier_date"), "yyyy-MM-dd HH:mm:ss"))
df_orders = df_orders.withColumn("order_delivered_customer_date", to_timestamp(col("order_delivered_customer_date"), "yyyy-MM-dd HH:mm:ss"))
df_orders = df_orders.withColumn("order_estimated_delivery_date", to_timestamp(col("order_estimated_delivery_date"), "yyyy-MM-dd HH:mm:ss"))


Tipos de dados: todas as colunas possuem o tipo de dados do objeto, possivelmente representando informações de string para order_id, customer_id, order_status. No entanto, para outras colunas, você deve usar o tipo de dados datetime porque representa o tempo.

Valor ausente: uma pequena porcentagem (<3%) de valores nulos está presente em order_approved_at, order_delivered_carrier_date e order_delivered_customer_date.

Sem valores negativos ou zero: não há valores negativos ou zero em todas as colunas porque todos os tipos de dados são objetos.

Duplicatas: não há linhas duplicadas no conjunto de dados, o que indica que cada linha representa uma revisão exclusiva.

## Dataset 'products'
Produtos : A tabela contém uma coleção de dados de produtos de plataformas de e-commerce que contém informações sobre vários produtos, como descrições ou tamanhos.
### Descrição das variáveis
![Produtos](img/dicionario/produtos.png)

### Limpeza e tratamento dos dados

In [46]:
print('linhas: ', df_products.count())
print('colunas: ', len(df_products.columns))

linhas:  32951
colunas:  9


In [47]:
df_products.describe().show(10)

+-------+--------------------+---------------------+-------------------+--------------------------+------------------+------------------+------------------+------------------+------------------+
|summary|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|  product_weight_g| product_length_cm| product_height_cm|  product_width_cm|
+-------+--------------------+---------------------+-------------------+--------------------------+------------------+------------------+------------------+------------------+------------------+
|  count|               32951|                32341|              32341|                     32341|             32341|             32949|             32949|             32949|             32949|
|   mean|                NULL|                 NULL|  48.47694876472589|         771.4952846232337|2.1889861166939797|2276.4724877841513| 30.81507784758263|16.937661234028347|23.196728277034204|
| stddev|                

In [48]:
df_products.show(10)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [49]:
df_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)



In [50]:
print(f'Contagem de valores nulos:')
for coluna in df_products.columns:
    print('Coluna:', coluna, '| Quantidade:', df_products.filter(df_products[coluna].isNull()).count())

print('\n')

print(f'Contagem de valores zerados:')
for coluna in df_products.columns:
    if isinstance(df_products.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_products.filter(df_products[coluna] == 0).count())

print('\n')

print(f'Contagem de valores negativos:')
for coluna in df_products.columns:
    if isinstance(df_products.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_products.filter(df_products[coluna] < 0).count())

Contagem de valores nulos:
Coluna: product_id | Quantidade: 0
Coluna: product_category_name | Quantidade: 610
Coluna: product_name_lenght | Quantidade: 610
Coluna: product_description_lenght | Quantidade: 610
Coluna: product_photos_qty | Quantidade: 610
Coluna: product_weight_g | Quantidade: 2
Coluna: product_length_cm | Quantidade: 2
Coluna: product_height_cm | Quantidade: 2
Coluna: product_width_cm | Quantidade: 2


Contagem de valores zerados:
Coluna: product_id | Quantidade: 0
Coluna: product_category_name | Quantidade: 0
Coluna: product_name_lenght | Quantidade: 0
Coluna: product_description_lenght | Quantidade: 0
Coluna: product_photos_qty | Quantidade: 0
Coluna: product_weight_g | Quantidade: 4
Coluna: product_length_cm | Quantidade: 0
Coluna: product_height_cm | Quantidade: 0
Coluna: product_width_cm | Quantidade: 0


Contagem de valores negativos:
Coluna: product_id | Quantidade: 0
Coluna: product_category_name | Quantidade: 0
Coluna: product_name_lenght | Quantidade: 0
Coluna

In [51]:
df_products = df_products.dropna()

Tipos de dados: Os dados consistem em dois tipos de dados principais, que são object e float64.

Valor ausente: a tabela mostra que a maioria das colunas tem uma baixa porcentagem de valores ausentes (null_value <2%). Isso sugere que os dados estão quase completos.

Sem Negativo: Não há colunas com porcentagem de valores negativos. Isso indica que todos os valores nessas colunas não são negativos.

Valores Zero: Apenas uma coluna possui uma porcentagem baixa de valores zero (0_value <1%). Isto sugere que os dados nesta coluna específica raramente contêm valores zero.

Duplicatas: não há linhas duplicadas neste conjunto de dados.

Número de valores exclusivos (n_unique): a coluna product_id possui 32.951 valores exclusivos, enquanto a coluna product_category_name possui 73 valores exclusivos. Isso indica uma variação significativa nas categorias de produtos.

## Dataset 'sellers'
Vendedores : Esta tabela contém informações sobre vendedores registrados na plataforma de e-commerce. Cada linha fornece detalhes sobre um único vendedor.
### Descrição das variáveis
![Vendedores](img/dicionario/vendedor.png)

### Limpeza e tratamento dos dados

In [52]:
print('linhas: ', df_sellers.count())
print('colunas: ', len(df_sellers.columns))

linhas:  3095
colunas:  4


In [53]:
df_sellers.describe().show(10)

+-------+--------------------+----------------------+-----------+------------+
|summary|           seller_id|seller_zip_code_prefix|seller_city|seller_state|
+-------+--------------------+----------------------+-----------+------------+
|  count|                3095|                  3095|       3095|        3095|
|   mean|                NULL|    32291.059450726978|  4482255.0|        NULL|
| stddev|                NULL|     32713.45382950901|       NULL|        NULL|
|    min|0015a82c2db000af6...|                  1001|   04482255|          AC|
|    max|ffff564a4f9085cd2...|                 99730|      xaxim|          SP|
+-------+--------------------+----------------------+-----------+------------+



In [54]:
df_sellers.show(10)

+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
|c240c4061717ac180...|                 20920|   rio de janeiro|          RJ|
|e49c26c3edfa46d22...|                 55325|           brejao|          PE|
|1b938a7ec6ac5061a...|                 16304|        penapolis|          SP|
|768a86e36ad6aae3d...|                  1529|        sao paulo|          SP|
|ccc4bbb5f32a6ab2b...|                 80310|         curitiba|          PR|

In [55]:
df_sellers.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



In [56]:
print(f'Contagem de valores nulos:')
for coluna in df_sellers.columns:
    print('Coluna:', coluna, '| Quantidade:', df_sellers.filter(df_sellers[coluna].isNull()).count())

print('\n')

print(f'Contagem de valores zerados:')
for coluna in df_sellers.columns:
    if isinstance(df_sellers.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_sellers.filter(df_sellers[coluna] == 0).count())

print('\n')

print(f'Contagem de valores negativos:')
for coluna in df_sellers.columns:
    if isinstance(df_sellers.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_sellers.filter(df_sellers[coluna] < 0).count())

Contagem de valores nulos:
Coluna: seller_id | Quantidade: 0
Coluna: seller_zip_code_prefix | Quantidade: 0
Coluna: seller_city | Quantidade: 0
Coluna: seller_state | Quantidade: 0


Contagem de valores zerados:
Coluna: seller_id | Quantidade: 0
Coluna: seller_zip_code_prefix | Quantidade: 0
Coluna: seller_city | Quantidade: 0
Coluna: seller_state | Quantidade: 0


Contagem de valores negativos:
Coluna: seller_id | Quantidade: 0
Coluna: seller_zip_code_prefix | Quantidade: 0
Coluna: seller_city | Quantidade: 0
Coluna: seller_state | Quantidade: 0


Valores ausentes: o conjunto de dados contém valores ausentes, mas eles são relativamente pequenos, representando menos de 2% do conjunto total de dados. Como resultado, uma abordagem para lidar com esses valores ausentes é simplesmente eliminar as linhas com valores ausentes.

Valores negativos: não há valores negativos presentes no conjunto de dados, o que é um sinal positivo, pois valores negativos em características como preço ou pagamento indicariam erros ou anomalias nos dados.

Valores Zero: O recurso frete_valor e pagamento_valor têm valores zero de 0,3% e 0,003%, respectivamente. Estas ocorrências podem ser consideradas normais, pois alguns pedidos podem incluir frete grátis ou pagamentos por utilização de vouchers ou promoções específicas.

Valores duplicados: não há valores duplicados no conjunto de dados. Cada registro é único, o que sugere que não há entradas idênticas ou repetições de dados no conjunto de dados.

Informações duplicadas: Existem duas colunas, nomeadamente product_category_name e product_category_name_english, contendo informações semelhantes sobre categorias de produtos. Podemos optar por utilizar um deles, pois fornecem informações redundantes.

## Dataset 'names'
Categorias : A tabela contém um conjunto de dados de tradução de nomes de categorias de produtos, usado para traduzir os nomes das categorias de produtos do português para o inglês.
### Analisando dados de Categorias
![Categorias](img/dicionario/categoria.png)

### Limpeza e tratamento dos dados

In [57]:
print('linhas: ', df_sellers.count())
print('colunas: ', len(df_sellers.columns))

linhas:  3095
colunas:  4


In [58]:
df_names.describe().show(10)

+-------+---------------------+-----------------------------+
|summary|product_category_name|product_category_name_english|
+-------+---------------------+-----------------------------+
|  count|                   71|                           71|
|   mean|                 NULL|                         NULL|
| stddev|                 NULL|                         NULL|
|    min| agro_industria_e_...|         agro_industry_and...|
|    max| utilidades_domest...|                watches_gifts|
+-------+---------------------+-----------------------------+



In [59]:
df_names.show(10)

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|         beleza_saude|                health_beauty|
| informatica_acess...|         computers_accesso...|
|           automotivo|                         auto|
|      cama_mesa_banho|               bed_bath_table|
|     moveis_decoracao|              furniture_decor|
|        esporte_lazer|               sports_leisure|
|           perfumaria|                    perfumery|
| utilidades_domest...|                   housewares|
|            telefonia|                    telephony|
|   relogios_presentes|                watches_gifts|
+---------------------+-----------------------------+
only showing top 10 rows



In [60]:
df_names.printSchema()

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)



In [61]:
print(f'Contagem de valores nulos:')
for coluna in df_names.columns:
    print('Coluna:', coluna, '| Quantidade:', df_names.filter(df_names[coluna].isNull()).count())

print('\n')

print(f'Contagem de valores zerados:')
for coluna in df_names.columns:
    if isinstance(df_names.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_names.filter(df_names[coluna] == 0).count())

print('\n')

print(f'Contagem de valores negativos:')
for coluna in df_names.columns:
    if isinstance(df_names.schema[coluna].dataType, TimestampType):
        continue
    print('Coluna:', coluna, '| Quantidade:', df_names.filter(df_names[coluna] < 0).count())

Contagem de valores nulos:
Coluna: product_category_name | Quantidade: 0
Coluna: product_category_name_english | Quantidade: 0


Contagem de valores zerados:
Coluna: product_category_name | Quantidade: 0
Coluna: product_category_name_english | Quantidade: 0


Contagem de valores negativos:
Coluna: product_category_name | Quantidade: 0
Coluna: product_category_name_english | Quantidade: 0


In [62]:
spark.stop()